In [1]:
import pandas as pd
import numpy as np
import time
import os
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pickle
import re

from selenium.common.exceptions import StaleElementReferenceException

chrome_options = Options()
chrome_options.add_argument('--headless')  # 웹 브라우저를 표시하지 않고 백그라운드에서 실행
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# 크롤링 전, 푼 문제가 100개 이하의 유저들은 제외.
(이미 제거된 상태)

In [2]:
#전처리 코드

df = pd.read_csv('..\DataSet\solved.ac_유저_크롤링.csv')
df

,user_name,user_tier,user_spc
0,teddybear1024,21,714
1,vladimir11,21,1162
2,minkim3987,21,238
3,movie_jo,21,3027
4,mymasterpark2,21,1062
...,...,...,...
32513,rsdfq1230,10,129
32514,seohee0419,10,336
32515,seun785,10,160
32516,sieglcg,10,175


In [3]:

with open('Baekjoon_user_list.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

submit_number_list = loaded_data['submit_number_list']
user_id_list = loaded_data['user_id_list']
problem_number_list = loaded_data['problem_number_list']
problem_result_list = loaded_data['problem_result_list']
code_memory_list = loaded_data['code_memory_list']
code_time_list = loaded_data['code_time_list']
code_language_list = loaded_data['code_language_list']
#code_length_list = loaded_data['code_length_list']


In [7]:


#Pickle 파일 망가졌을 때, 기존 데이터셋 가져와서 크롤링

r_data = pd.read_csv('../DataSet/Baekjoon_유저_크롤링.csv')

submit_number_list = list(r_data['submit_number_list'])
user_id_list = list(r_data['user_id_list'])
problem_number_list = list(r_data['problem_number_list'])
problem_result_list = list(r_data['problem_result_list'])
code_memory_list = list(r_data['code_memory_list'])
code_time_list = list(r_data['code_time_list'])
code_language_list = list(r_data['code_language_list'])



In [4]:
"""
print(submit_number_list)
print(user_id_list)
print(problem_number_list)
print(problem_result_list)
print(code_memory_list)
print(code_time_list)
print(code_language_list)
print(code_length_list)
"""

print(len(submit_number_list))
print(len(user_id_list))
print(len(problem_number_list))
print(len(problem_result_list))
print(len(code_memory_list))
print(len(code_time_list))
print(len(code_language_list))

num=-1
print(submit_number_list[num])
print(user_id_list[num])
print(problem_number_list[num])
print(problem_result_list[num])
print(code_memory_list[num])
print(code_time_list[num])
print(code_language_list[num])


15314772
15314772
15314772
15314772
15314772
15314772
15314772
6670370
itlockit
2557
틀렸습니다


Java 8


In [5]:
list(df['user_name']).index('fabula')

14500

In [11]:
list(df['user_name'])[13640]

'da1907'

In [9]:
from selenium.common.exceptions import NoSuchElementException

#실행하기전에 무조건 base_num 바꾸고, 위에 100이하 걸러주는 코드 실행시키고 해라.

base_num = 14501 #시작해야하는 번호를 적으면 됨.


for u, user in enumerate(list(df['user_name'])):
    
    if(u<base_num):
        continue

    baekjoon_user_url = 'https://www.acmicpc.net/status?user_id=' + str(user)
    #baekjoon_user_url = 'https://www.acmicpc.net/status?user_id=' + 'llsy159'
    
    print(u, baekjoon_user_url)
    driver = webdriver.Chrome('chromedriver.exe', options=chrome_options)
    driver.get(baekjoon_user_url)

    sub_submit_number_list = []
    sub_user_id_list = []
    sub_problem_number_list = []
    sub_problem_result_list = []
    sub_code_memory_list = []
    sub_code_time_list = []
    sub_code_language_list = []
    #sub_code_length_list = []
    #sub_submit_time_list = []

    flag = True

    while(True):

        user_data = driver.find_elements_by_css_selector('[data-can-view]')
        #user_real_time = driver.find_elements_by_css_selector('.real-time-update')
        
        try:
            user_next_page = driver.find_element_by_css_selector('#next_page')

        except NoSuchElementException:
            flag = False

        """
        for urt in user_real_time:
            sub_submit_time_list.append(urt.get_attribute('data-original-title'))
        """
        
        #print(sub_submit_time_list)

        for user_info in user_data:
        
            result = user_info.find_element_by_css_selector('.result-text').text
            memory_text = user_info.find_element_by_css_selector('.memory').text
            time_text = user_info.find_element_by_css_selector('.time').text

            user_text = user_info.find_elements_by_css_selector("td")
            user_text_list = []

            for ut in user_text:
                user_text_list.append(ut.text)
            
            sub_submit_number_list.append(user_text_list[0])
            sub_user_id_list.append(user_text_list[1])
            sub_problem_number_list.append(user_text_list[2])
            sub_problem_result_list.append(user_text_list[3])
            sub_code_memory_list.append(user_text_list[4])
            sub_code_time_list.append(user_text_list[5])
            sub_code_language_list.append(user_text_list[6])
            #sub_code_length_list.append(user_text_list[7])

        if(flag):
            user_next_page.click()
        else:
            break            

    driver.quit()

    submit_number_list.extend(sub_submit_number_list)
    user_id_list.extend(sub_user_id_list)
    problem_number_list.extend(sub_problem_number_list)
    problem_result_list.extend(sub_problem_result_list)
    code_memory_list.extend(sub_code_memory_list)
    code_time_list.extend(sub_code_time_list)
    code_language_list.extend(sub_code_language_list)

    save_data = {
        'submit_number_list': submit_number_list,
        'user_id_list': user_id_list,
        'problem_number_list': problem_number_list,
        'problem_result_list': problem_result_list,
        'code_memory_list': code_memory_list,
        'code_time_list': code_time_list,
        'code_language_list': code_language_list,
    }

    with open('Baekjoon_user_list.pkl', 'wb') as f:
        pickle.dump(save_data, f)
    
    if(u%500 == 0):
        result_df = pd.DataFrame(save_data) 
        result_df.to_csv('..\DataSet\Baekjoon_유저_크롤링.csv',index=False)
    


14501 https://www.acmicpc.net/status?user_id=flqms649
14502 https://www.acmicpc.net/status?user_id=g22206
14503 https://www.acmicpc.net/status?user_id=gch06218
14504 https://www.acmicpc.net/status?user_id=gjxodud1
14505 https://www.acmicpc.net/status?user_id=gocks0203
14506 https://www.acmicpc.net/status?user_id=harry0558
14507 https://www.acmicpc.net/status?user_id=hyj6125
14508 https://www.acmicpc.net/status?user_id=jisue0216
14509 https://www.acmicpc.net/status?user_id=joori0709
14510 https://www.acmicpc.net/status?user_id=kodari385
14511 https://www.acmicpc.net/status?user_id=kyungsub1996
14512 https://www.acmicpc.net/status?user_id=lemongrabmayo
14513 https://www.acmicpc.net/status?user_id=lets_social_distance
14514 https://www.acmicpc.net/status?user_id=nandong1104
14515 https://www.acmicpc.net/status?user_id=padamhyunn
14516 https://www.acmicpc.net/status?user_id=shyunju7
14517 https://www.acmicpc.net/status?user_id=trillion
14518 https://www.acmicpc.net/status?user_id=wogks99
1

잘못 멈춰서 Pickle파일 손상되면, 에러 문구부터가 달라, 그거 잘 보고, 바로 리스트들 날라가기전에 csv파일로 저장해

In [ ]:
"""
잘못됐을때 
with open('Baekjoon_user_list.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

submit_number_list = loaded_data['submit_number_list']
user_id_list = loaded_data['user_id_list']
problem_number_list = loaded_data['problem_number_list']
problem_result_list = loaded_data['problem_result_list']
code_memory_list = loaded_data['code_memory_list']
code_time_list = loaded_data['code_time_list']
code_language_list = loaded_data['code_language_list']
code_length_list = loaded_data['code_length_list']
submit_time_list = loaded_data['submit_time_list']
"""


In [6]:
print(len(submit_number_list))
print(len(user_id_list))
print(len(problem_number_list))
print(len(problem_result_list))
print(len(code_memory_list))
print(len(code_time_list))
print(len(code_language_list))


num=-1

print(submit_number_list[num])
print(user_id_list[num])
print(problem_number_list[num])
print(problem_result_list[num])
print(code_memory_list[num])
print(code_time_list[num])
print(code_language_list[num])


14535329
14535329
14535329
14535329
14535329
14535329
14535329
57044532
fabula
2557
맞았습니다!!
1112
0
C++17


In [7]:
data = {'submit_number_list': submit_number_list,
        'user_id_list': user_id_list,
        'problem_number_list': problem_number_list,
        'problem_result_list': problem_result_list,
        'code_memory_list': code_memory_list,
        'code_time_list': code_time_list,
        'code_language_list': code_language_list,
    }

result_df = pd.DataFrame(data)
result_df

,submit_number_list,user_id_list,problem_number_list,problem_result_list,code_memory_list,code_time_list,code_language_list
0,66660086,teddybear1024,12920.0,맞았습니다!!,60700.0,72.0,C++17
1,66328551,teddybear1024,29768.0,맞았습니다!!,2020.0,0.0,C++17
2,66328354,teddybear1024,29767.0,맞았습니다!!,4364.0,96.0,C++17
3,66328329,teddybear1024,29767.0,틀렸습니다,-1.0,-1.0,C++17
4,65610566,teddybear1024,13511.0,맞았습니다!!,47144.0,244.0,C++17
...,...,...,...,...,...,...,...
14535324,57045020,fabula,1008,틀렸습니다,,,C++17
14535325,57044678,fabula,10998,맞았습니다!!,1112,0,C++17
14535326,57044660,fabula,1001,맞았습니다!!,1112,0,C++17
14535327,57044593,fabula,1000,맞았습니다!!,1112,0,C++17


In [8]:
result_df.to_csv('..\DataSet\Baekjoon_유저_크롤링.csv',index=False)